In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
song_cluster_scores = pd.read_csv('../data/song_cluster_scores.csv', index_col='track_id')

In [34]:
example_face_vectors = {
    "angry":    [0.241, 0.000, 0.009, 0.008, 0.699, 0.042, 0.002],
    "disgust":  [0.414, 0.469, 0.013, 0.002, 0.074, 0.022, 0.005],
    "fear":     [0.186, 0.004, 0.255, 0.009, 0.186, 0.351, 0.009],
    "happy":    [0.000, 0.000, 0.000, 0.995, 0.002, 0.000, 0.002],
    "neutral":  [0.004, 0.000, 0.005, 0.036, 0.929, 0.026, 0.000],
    "sad":      [0.050, 0.001, 0.089, 0.000, 0.245, 0.613, 0.001],
    "surprise": [0.007, 0.000, 0.008, 0.019, 0.028, 0.002, 0.937]
}

In [93]:
# Example PANAS outputs
example_panas = {
    "Stressed": [0.80, 0.65, 0.10, 0.15, 0.20, 0.25, 0.70],  # 1. High stress/anger → tense or aggressive music
    "Angry":    [0.05, 0.03, 0.95, 0.10, 0.85, 0.90, 0.08],  # 2. Joyful/excited → upbeat or danceable
    "Sad":      [0.10, 0.05, 0.20, 0.80, 0.30, 0.25, 0.60],  # 3. Sad/nervous → calm or reflective
    "Happy":    [0.15, 0.10, 0.60, 0.20, 0.90, 0.80, 0.10],  # 4. Inspired + happy → motivational
    "Neutral":  [0.40, 0.35, 0.30, 0.50, 0.50, 0.45, 0.50],  # 5. Mixed mood → neutral playlist
    "Excited":  [0.10, 0.05, 0.10, 0.05, 0.95, 0.80, 0.05],  # 6. Very inspired/excited → energetic + meaningful
    "Nervous":  [0.60, 0.40, 0.15, 0.70, 0.10, 0.15, 0.75],  # 7. Anxious/sad → moody or ambient
}

In [90]:
# PANAS order:
PANAS_LABELS = ['Stressed', 'Angry', 'Happy', 'Sad', 'Inspired', 'Excited', 'Nervous']

# Original facial CNN order:
FACIAL_LABELS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Mapping facial to PANAS positions
def remap_face_to_panas(face_vec):
    return np.array([
        face_vec[1],  # Stressed     ← disgust
        face_vec[0],  # Angry        ← angry
        face_vec[3],  # Happy        ← happy
        face_vec[5],  # Sad          ← sad
        face_vec[4],  # Inspired     ← no direct match
        face_vec[6],  # Excited      ← surprise
        face_vec[2]   # Nervous      ← fear
    ])

# Final combine function
def get_combined_emotion_vector(face_vec, panas_vec_raw):
    # === Step 1: Remap facial emotion to PANAS-aligned 7D
    face_panas_vec = remap_face_to_panas(face_vec)
    
    # === Step 2: Normalize each vector
    face_norm = face_panas_vec / (np.sum(face_panas_vec) + 1e-8)
    
    # PANAS: scale from [1, 5] to [0, 1]
    panas_norm = (panas_vec_raw - 1.0) / 4.0

    # === Step 3: Stack into single input vector
    combined_vec = np.hstack([face_norm, panas_norm])
    return combined_vec  # shape: (14,)

In [91]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def recommend_playlist_from_emotion( 
    face_vec, panas_vec, song_cluster_scores, top_n=10
):
    """
    Args:
        face_vec: raw 7-d output from facial emotion model
        panas_vec: raw 7-d PANAS scores from biometric model (1-5 scale)
        song_cluster_scores: DataFrame with 7-d vibe vectors per song (index = track_id)
        top_n: number of songs to return

    Returns:
        DataFrame of top N recommended track_ids and their similarity scores
    """
    combined_vec = get_combined_emotion_vector(face_vec, panas_vec)
    user_emotion = combined_vec[-7:]
    user_emotion = user_emotion / (np.linalg.norm(user_emotion) + 1e-8)

    song_vecs = song_cluster_scores.values
    song_vecs = song_vecs / (np.linalg.norm(song_vecs, axis=1, keepdims=True) + 1e-8)

    similarities = cosine_similarity(song_vecs, user_emotion.reshape(1, -1)).flatten()
    top_idx = similarities.argsort()[::-1][:top_n]

    return pd.DataFrame({
        "track_id": song_cluster_scores.index[top_idx],
        "Similarity": similarities[top_idx]
    }).reset_index(drop=True)

In [73]:
face_pred_happy = [0.0, 0.0, 0.0, 0.995, 0.002, 0.0, 0.002]

face_pred_surprise = [0.007, 0.0, 0.008, 0.019, 0.028, 0.002, 0.937]

face_pred_sad = [0.05, 0.001, 0.089, 0.0, 0.245, 0.613, 0.001]

face_pred_angry = [0.241, 0.0, 0.009, 0.008, 0.699, 0.042, 0.002]

face_pred_fear = [0.186, 0.004, 0.255, 0.009, 0.186, 0.351, 0.009]

In [74]:
panas_pred_stressed_nervous = [
    0.00653729, -0.18582821, -0.23318774, -0.2487897,
    -0.23372887, -0.12444461, 0.00557272
]

panas_pred_excited_stressed = [
    -0.14335369, -0.2734212, -0.21269707, -0.26963243,
    -0.22057566, -0.07807975, -0.14732485
]

panas_pred_happy_excited_inspired = [
    -0.25548002, -0.29832116, -0.25125208, -0.28806356,
    -0.24982394, -0.24082947, -0.2577177
]

In [75]:
songs_info_df = pd.read_csv('../data/SONGS.csv')

In [86]:
face_vec = np.array(face_pred_happy)  # CNN softmax output
panas_vec = np.array(panas_pred_happy_excited_inspired)   # biometric PANAS (1-5 scale)

playlist_df = recommend_playlist_from_emotion(
    face_vec=face_vec,
    panas_vec=panas_vec,
    song_cluster_scores=song_cluster_scores,
    top_n=20
)

# Clean track_id to get just the integer part before " - "
playlist_df["track_id_int"] = playlist_df["track_id"].str.extract(r"^(\d+)", expand=False).astype(int)

# Merge with the full song info table on index
playlist_merged = playlist_df.merge(
    songs_info_df,
    left_on="track_id_int",
    right_index=True,
    how="left"
)

# Create combined label
playlist_merged["Song"] = playlist_merged["Track Name"] + " - " + playlist_merged["Artist"]

# Optional: reorder and select final columns
final_playlist = playlist_merged[["Song", "Similarity"]]

# Show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
print(final_playlist)

                                                                                          Song  \
0                                                  The Door - CYRIL Remix - Teddy Swims, CYRIL   
1                                      Kisses (feat. bbyclose) - BL3SS, CamrinWatsin, bbyclose   
2                                                                              BATTITO - Fedez   
3   Call Me When You Break Up (with Gracie Abrams) - Selena Gomez, benny blanco, Gracie Abrams   
4                                                                         505 - Arctic Monkeys   
5                                                          MILLION DOLLAR BABY - Tommy Richman   
6                                                                             sobelove - Beéle   
7                            GOOD CREDIT (with Kendrick Lamar) - Playboi Carti, Kendrick Lamar   
8                                                   Scream & Shout - will.i.am, Britney Spears   
9                   

In [87]:
face_vec = np.array(face_pred_surprise)  # CNN softmax output
panas_vec = np.array(panas_pred_excited_stressed)   # biometric PANAS (1-5 scale)

playlist_df = recommend_playlist_from_emotion(
    face_vec=face_vec,
    panas_vec=panas_vec,
    song_cluster_scores=song_cluster_scores,
    top_n=20
)

# Clean track_id to get just the integer part before " - "
playlist_df["track_id_int"] = playlist_df["track_id"].str.extract(r"^(\d+)", expand=False).astype(int)

# Merge with the full song info table on index
playlist_merged = playlist_df.merge(
    songs_info_df,
    left_on="track_id_int",
    right_index=True,
    how="left"
)

# Create combined label
playlist_merged["Song"] = playlist_merged["Track Name"] + " - " + playlist_merged["Artist"]

# Optional: reorder and select final columns
final_playlist = playlist_merged[["Song", "Similarity"]]

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
final_playlist

,Song,Similarity
0,"The Door - CYRIL Remix - Teddy Swims, CYRIL",0.808712
1,"Kisses (feat. bbyclose) - BL3SS, CamrinWatsin, bbyclose",0.804731
2,BATTITO - Fedez,0.799808
3,"Call Me When You Break Up (with Gracie Abrams) - Selena Gomez, benny blanco, Gracie Abrams",0.799041
4,Diet Pepsi - Addison Rae,0.767222
5,MILLION DOLLAR BABY - Tommy Richman,0.764297
6,505 - Arctic Monkeys,0.761427
7,sobelove - Beéle,0.756356
8,"Scream & Shout - will.i.am, Britney Spears",0.751590
9,"GOOD CREDIT (with Kendrick Lamar) - Playboi Carti, Kendrick Lamar",0.750031


In [88]:
face_vec = np.array(panas_pred_stressed_nervous)  # CNN softmax output
panas_vec = np.array(face_pred_fear)   # biometric PANAS (1-5 scale)

playlist_df = recommend_playlist_from_emotion(
    face_vec=face_vec,
    panas_vec=panas_vec,
    song_cluster_scores=song_cluster_scores,
    top_n=20
)

# Clean track_id to get just the integer part before " - "
playlist_df["track_id_int"] = playlist_df["track_id"].str.extract(r"^(\d+)", expand=False).astype(int)

# Merge with the full song info table on index
playlist_merged = playlist_df.merge(
    songs_info_df,
    left_on="track_id_int",
    right_index=True,
    how="left"
)

# Create combined label
playlist_merged["Song"] = playlist_merged["Track Name"] + " - " + playlist_merged["Artist"]

# Optional: reorder and select final columns
final_playlist = playlist_merged[["Song", "Similarity"]]

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
final_playlist

,Song,Similarity
0,BATTITO - Fedez,0.847726
1,"The Door - CYRIL Remix - Teddy Swims, CYRIL",0.835113
2,"Kisses (feat. bbyclose) - BL3SS, CamrinWatsin, bbyclose",0.820183
3,"Call Me When You Break Up (with Gracie Abrams) - Selena Gomez, benny blanco, Gracie Abrams",0.808726
4,"Scream & Shout - will.i.am, Britney Spears",0.802241
5,505 - Arctic Monkeys,0.799782
6,sobelove - Beéle,0.799525
7,"Chiquita - Neton Vega, Tito Double P",0.789706
8,"ANXIETY (feat. Doechii) - Sleepy Hallow, Doechii",0.788605
9,I Wonder - Kanye West,0.787646


In [ ]:
face_vec = np.array(face_pred_sad)  # CNN softmax output
panas_vec = np.array(panas_pred_stressed_nervous)   # biometric PANAS (1-5 scale)

playlist_df = recommend_playlist_from_emotion(
    face_vec=face_vec,
    panas_vec=panas_vec,
    song_cluster_scores=song_cluster_scores,
    top_n=20
)

# Clean track_id to get just the integer part before " - "
playlist_df["track_id_int"] = playlist_df["track_id"].str.extract(r"^(\d+)", expand=False).astype(int)

# Merge with the full song info table on index
playlist_merged = playlist_df.merge(
    songs_info_df,
    left_on="track_id_int",
    right_index=True,
    how="left"
)

# Create combined label
playlist_merged["Song"] = playlist_merged["Track Name"] + " - " + playlist_merged["Artist"]

# Optional: reorder and select final columns
final_playlist = playlist_merged[["Song", "Similarity"]]

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
final_playlist

,Song,Similarity
0,Call Me When You Break Up (with Gracie Abrams)...,0.791542
1,"The Door - CYRIL Remix - Teddy Swims, CYRIL",0.788080
2,"Kisses (feat. bbyclose) - BL3SS, CamrinWatsin,...",0.783358
3,BATTITO - Fedez,0.765654
4,MILLION DOLLAR BABY - Tommy Richman,0.748385
5,Diet Pepsi - Addison Rae,0.745779
6,GOOD CREDIT (with Kendrick Lamar) - Playboi Ca...,0.733724
7,sobelove - Beéle,0.733526
8,505 - Arctic Monkeys,0.731519
9,"Scream & Shout - will.i.am, Britney Spears",0.722433


In [96]:
face_vec = np.array(example_face_vectors['surprise'])  # CNN softmax output
panas_vec = np.array(example_panas['Excited'])   # biometric PANAS (1-5 scale)

playlist_df = recommend_playlist_from_emotion(
    face_vec=face_vec,
    panas_vec=panas_vec,
    song_cluster_scores=song_cluster_scores,
    top_n=21
)

# Clean track_id to get just the integer part before " - "
playlist_df["track_id_int"] = playlist_df["track_id"].str.extract(r"^(\d+)", expand=False).astype(int)

# Merge with the full song info table on index
playlist_merged = playlist_df.merge(
    songs_info_df,
    left_on="track_id_int",
    right_index=True,
    how="left"
)

# Create combined label
playlist_merged["Song"] = playlist_merged["Track Name"] + " - " + playlist_merged["Artist"]

# Optional: reorder and select final columns
final_playlist = playlist_merged[["Song", "Similarity"]]

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
final_playlist

,Song,Similarity
0,"Ni**as In Paris - JAY-Z, Kanye West",0.848737
1,Too Much - Dove Cameron,0.847178
2,"FXCK UP THE WORLD (feat. Future) - LISA, Future",0.842028
3,"I Smoked Away My Brain (I'm God x Demons Mashup) (feat. Imogen Heap & Clams Casino) - A$AP Rocky, Imogen Heap, Clams Casino",0.840972
4,Cópia Proibida - Léo Foguete,0.839458
5,"Kiss Me Thru The Phone - Soulja Boy, Sammie",0.836395
6,Under Your Spell - Snow Strippers,0.830006
7,FOMDJ - Playboi Carti,0.827926
8,The Door - Teddy Swims,0.827648
9,"blackout 🧊 - Emilia, TINI, Nicki Nicole",0.814923


In [84]:
face_vec = np.array(example_face_vectors['happy'])  # CNN softmax output
panas_vec = np.array(example_panas['Happy'])   # biometric PANAS (1-5 scale)

playlist_df = recommend_playlist_from_emotion(
    face_vec=face_vec,
    panas_vec=panas_vec,
    song_cluster_scores=song_cluster_scores,
    top_n=20
)

# Clean track_id to get just the integer part before " - "
playlist_df["track_id_int"] = playlist_df["track_id"].str.extract(r"^(\d+)", expand=False).astype(int)

# Merge with the full song info table on index
playlist_merged = playlist_df.merge(
    songs_info_df,
    left_on="track_id_int",
    right_index=True,
    how="left"
)

# Create combined label
playlist_merged["Song"] = playlist_merged["Track Name"] + " - " + playlist_merged["Artist"]

# Optional: reorder and select final columns
final_playlist = playlist_merged[["Song", "Similarity"]]

final_playlist

,Song,Similarity
0,FOMDJ - Playboi Carti,0.900789
1,Too Much - Dove Cameron,0.895781
2,"Something About You - Eyedress, Dent May",0.895760
3,"ANXIETY (feat. Doechii) - Sleepy Hallow, Doechii",0.885084
4,The Door - Teddy Swims,0.879170
5,Waiting For Love - Avicii,0.877040
6,Cópia Proibida - Léo Foguete,0.874364
7,"Quevedo: Bzrp Music Sessions, Vol. 52 - Bizarr...",0.873995
8,I Wonder - Kanye West,0.871488
9,"blackout 🧊 - Emilia, TINI, Nicki Nicole",0.871315
